# Blobplot contamination detection and removal

Hifiams assemblies from triobinned reads (including unbinned reads (~2%)). We chose the purge_duped assemblies. 

In [12]:
WKDIR=/workspace/hraijc/Blueberry/Blueberry_trio/Assembly
ASSEMBLYDIR=/output/genomic/plant/Vaccinium/corymbosum/2023-10-18_M7xNui_TrioBinned_HiFi_Assemblies/Classified_Reads_Plus_Unclassified_Reads_Assemblies
READDIR=${WKDIR}/HiFiReads
LOG=${WKDIR}/log

cd ${WKDIR}

In [2]:
mkdir -p ${LOG}
mkdir -p ${WKDIR}/precontam
mkdir -p ${READDIR}
mkdir -p ${WKDIR}/bamfiles
mkdir -p ${WKDIR}/blasthits
mkdir -p ${WKDIR}/BlobPlot_out


In [32]:
# Symlink Hifiams assemblies from triobinned reads (including unbinned reads (~2%)). We chose the purge_duped assemblies. 
ln -s ${ASSEMBLYDIR}/Nui_Haplotypes/Purged_Assemblies/classified_Nui_plus_unclassified_hap2.purged.fa ${WKDIR}/precontam/
ln -s ${ASSEMBLYDIR}/Nui_Haplotypes/Purged_Assemblies/classified_Nui_plus_unclassified_hap1.purged.fa ${WKDIR}/precontam/
ln -s ${ASSEMBLYDIR}/M7_Haplotypes/Purged_Assemblies/classified_M7_plus_unclassified_hap1.purged.fa ${WKDIR}/precontam/
ln -s ${ASSEMBLYDIR}/M7_Haplotypes/Purged_Assemblies/classified_M7_plus_unclassified_hap2.purged.fa ${WKDIR}/precontam/

ln: failed to create symbolic link ‘/workspace/hraijc/Blueberry/Blueberry_trio/Assembly/precontam/classified_Nui_plus_unclassified_hap2.purged.fa’: File exists
ln: failed to create symbolic link ‘/workspace/hraijc/Blueberry/Blueberry_trio/Assembly/precontam/classified_Nui_plus_unclassified_hap1.purged.fa’: File exists
ln: failed to create symbolic link ‘/workspace/hraijc/Blueberry/Blueberry_trio/Assembly/precontam/classified_M7_plus_unclassified_hap1.purged.fa’: File exists
ln: failed to create symbolic link ‘/workspace/hraijc/Blueberry/Blueberry_trio/Assembly/precontam/classified_M7_plus_unclassified_hap2.purged.fa’: File exists


: 1

# Map the reads back to Assembly for Blobtools coverage

In [ ]:
# Decompress HiFi reads into a directory in my workspace.
for readfile in /workspace/hrasrb/Blueberry_trio/rerun/trio-binning/02.classifyHiFi/*unclassified.fasta.gz
do
    readfilename=$(basename "$readfile")
    echo gzip -dc ${readfile} > ${READDIR}/${readfilename%.gz}
done | abatch -j ${LOG} --time 0:30:00 --cpus-per-task=1 --mem 1G | sbatch

In [8]:
# Script to map and create sorted bamfile for each parent. 
cat  ${WKDIR}/log/map.sh

echo "#minimap2 -t 8 -ax map-hifi ${assemb} ${reads} > ${WKDIR}/bamfiles/${assembfilename}_to_hifi.sam"
echo "samtools view --threads 8 -b ${WKDIR}/bamfiles/${assembfilename}_to_hifi.sam > ${WKDIR}/bamfiles/${assembfilename}_to_hifi.bam"
echo "samtools sort --threads 8 ${WKDIR}/bamfiles/${assembfilename}_to_hifi.bam > ${WKDIR}/bamfiles/${assembfilename}_to_hifi.sorted.bam "
echo "samtools index ${WKDIR}/bamfiles/${assembfilename}_to_hifi.sorted.bam"

In [9]:
for assemb in ${WKDIR}/precontam/*.fa
do
    if [[ $assemb == *"M7"* ]]; then
        PARENT=M7
    else
        PARENT=Nui
    fi
    export WKDIR=${WKDIR}
    export assemb=${assemb}
    export reads=${READDIR}/classified_${PARENT}_plus_unclassified.fasta
    export assembfilename=$(basename "$assemb" .purged.fa)
    sh ${WKDIR}/log/map.sh > ${WKDIR}/log/${assembfilename}_map.sh
    
done

In [10]:
cat ${WKDIR}/log/classified_M7_plus_unclassified_hap2_map.sh

#minimap2 -t 8 -ax map-hifi /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/precontam/classified_M7_plus_unclassified_hap2.purged.fa /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/HiFiReads/classified_M7_plus_unclassified.fasta > /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/bamfiles/classified_M7_plus_unclassified_hap2_to_hifi.sam
samtools view --threads 8 -b /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/bamfiles/classified_M7_plus_unclassified_hap2_to_hifi.sam > /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/bamfiles/classified_M7_plus_unclassified_hap2_to_hifi.bam
samtools sort --threads 8 /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/bamfiles/classified_M7_plus_unclassified_hap2_to_hifi.bam > /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/bamfiles/classified_M7_plus_unclassified_hap2_to_hifi.sorted.bam 
samtools index /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/bamfiles/classified_M7_plus_unclassified_hap2_to_hifi.sorted.bam


In [13]:
for shellfile in ${WKDIR}/log/*_map.sh
do
    echo "sh $shellfile"
done > $WKDIR/log/map2.sh 



In [14]:
cat $WKDIR/log/map2.sh 

sh /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/log/classified_M7_plus_unclassified_hap1_map.sh
sh /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/log/classified_M7_plus_unclassified_hap2_map.sh
sh /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/log/classified_Nui_plus_unclassified_hap1_map.sh
sh /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/log/classified_Nui_plus_unclassified_hap2_map.sh


In [15]:
module load minimap2
module load samtools

cat $WKDIR/log/map2.sh | abatch -j minimap --time 10:00:00 --mem 12G --cpus-per-task=8 | sbatch

module unload samtools
module unload minimap2

SBATCH_ARGS: --time 10:00:00 --mem 12G --cpus-per-task=8
JOB_ARRAY_NAME: minimap
GROUP_SIZE: 1
NUM_COMMANDS: 4
Submitted batch job 2140586


In [16]:
seff 2140586

Job ID: 2140586
Array Job ID: 2140586_4
Cluster: powerplant
User/Group: hraijc/hraijc
State: RUNNING
Nodes: 1
Cores per node: 8
CPU Utilized: 00:00:00
CPU Efficiency: 0.00% of 00:07:28 core-walltime
Job Wall-clock time: 00:00:56
Memory Utilized: 0.00 MB (estimated maximum)
Memory Efficiency: 0.00% of 12.00 GB (12.00 GB/node)


## generate blast result

In [66]:
module load ncbi-blast

for file in ${WKDIR}/precontam/*.fa
do
    filename=$(basename "$file" .purged.fa)
    echo "blastn \
    -task megablast \
    -query "${file}" \
    -db /workspace/ComparativeDataSources/NCBI/nt/nt \
    -outfmt '6 qseqid staxids bitscore std' \
    -evalue 1e-10 \
    -num_threads 12 | sort -k1,1 -k3,3nr -k14,14n | sort -u -k1,1 --merge > "${WKDIR}/blasthits/${filename}.vs.nt.best_single.1e10.megablast.out""
done | abatch -j MegaBlast --time 20:00:00 --mem 2G --cpus-per-task=12 | sbatch

module unload ncbi-blast


SBATCH_ARGS: --time 20:00:00 --mem 2G --cpus-per-task=12
JOB_ARRAY_NAME: MegaBlast
GROUP_SIZE: 1
NUM_COMMANDS: 4
Submitted batch job 2140270


In [17]:
seff 2140270


Job ID: 2140270
Array Job ID: 2140270_4
Cluster: powerplant
User/Group: hraijc/hraijc
State: COMPLETED (exit code 0)
Nodes: 1
Cores per node: 12
CPU Utilized: 22:36:15
CPU Efficiency: 90.00% of 1-01:07:00 core-walltime
Job Wall-clock time: 02:05:35
Memory Utilized: 1.04 GB
Memory Efficiency: 52.11% of 2.00 GB


## generate blobplot

In [3]:
for file in "${WKDIR}/precontam"/*.fa
do
    filename=$(basename "$file" .purged.fa)
    echo "blobtools create -i $file -b ${WKDIR}/bamfiles/${filename}_to_hifi.sorted.bam -t ${WKDIR}/blasthits/${filename}.vs.nt.best_single.1e10.megablast.out -o ${WKDIR}/BlobPlot_out/${filename}.best_single.blobplot"
    echo "blobtools plot -i ${WKDIR}/BlobPlot_out/${filename}.best_single.blobplot.blobDB.json -o ${WKDIR}/BlobPlot_out/${filename}.best_single.blobplot"
    echo "blobtools view -i ${WKDIR}/BlobPlot_out/${filename}.best_single.blobplot.blobDB.json -o ${WKDIR}/BlobPlot_out/${filename}.best_single.blobplot"

done 

blobtools create -i /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/precontam/classified_M7_plus_unclassified_hap1.purged.fa -b /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/bamfiles/classified_M7_plus_unclassified_hap1_to_hifi.sorted.bam -t /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/blasthits/classified_M7_plus_unclassified_hap1.vs.nt.best_single.1e10.megablast.out -o /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/BlobPlot_out/classified_M7_plus_unclassified_hap1.best_single.blobplot
blobtools plot -i /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/BlobPlot_out/classified_M7_plus_unclassified_hap1.best_single.blobplot.blobDB.json -o /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/BlobPlot_out/classified_M7_plus_unclassified_hap1.best_single.blobplot
blobtools view -i /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/BlobPlot_out/classified_M7_plus_unclassified_hap1.best_single.blobplot.blobDB.json -o /workspace/hraijc/Blueberry/Blueberry_trio/Assembly

In [6]:
# Run Blobtools. Copying output from previous cell. 
sbatch << EOF
#!/bin/bash
#SBATCH -J blobtools
#SBATCH -o ${LOG}/%J.out
#SBATCH -e ${LOG}/%J.err
#SBATCH --cpus-per-task=1
#SBATCH --mem=1G
#SBATCH --time=2:00:00

module load conda
conda deactivate
module load blobtools

blobtools create -i /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/precontam/classified_M7_plus_unclassified_hap1.purged.fa -b /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/bamfiles/classified_M7_plus_unclassified_hap1_to_hifi.sorted.bam -t /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/blasthits/classified_M7_plus_unclassified_hap1.vs.nt.best_single.1e10.megablast.out -o /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/BlobPlot_out/classified_M7_plus_unclassified_hap1.best_single.blobplot
blobtools plot -i /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/BlobPlot_out/classified_M7_plus_unclassified_hap1.best_single.blobplot.blobDB.json -o /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/BlobPlot_out/classified_M7_plus_unclassified_hap1.best_single.blobplot
blobtools view -i /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/BlobPlot_out/classified_M7_plus_unclassified_hap1.best_single.blobplot.blobDB.json -o /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/BlobPlot_out/classified_M7_plus_unclassified_hap1.best_single.blobplot
blobtools create -i /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/precontam/classified_M7_plus_unclassified_hap2.purged.fa -b /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/bamfiles/classified_M7_plus_unclassified_hap2_to_hifi.sorted.bam -t /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/blasthits/classified_M7_plus_unclassified_hap2.vs.nt.best_single.1e10.megablast.out -o /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/BlobPlot_out/classified_M7_plus_unclassified_hap2.best_single.blobplot
blobtools plot -i /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/BlobPlot_out/classified_M7_plus_unclassified_hap2.best_single.blobplot.blobDB.json -o /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/BlobPlot_out/classified_M7_plus_unclassified_hap2.best_single.blobplot
blobtools view -i /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/BlobPlot_out/classified_M7_plus_unclassified_hap2.best_single.blobplot.blobDB.json -o /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/BlobPlot_out/classified_M7_plus_unclassified_hap2.best_single.blobplot
blobtools create -i /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/precontam/classified_Nui_plus_unclassified_hap1.purged.fa -b /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/bamfiles/classified_Nui_plus_unclassified_hap1_to_hifi.sorted.bam -t /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/blasthits/classified_Nui_plus_unclassified_hap1.vs.nt.best_single.1e10.megablast.out -o /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/BlobPlot_out/classified_Nui_plus_unclassified_hap1.best_single.blobplot
blobtools plot -i /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/BlobPlot_out/classified_Nui_plus_unclassified_hap1.best_single.blobplot.blobDB.json -o /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/BlobPlot_out/classified_Nui_plus_unclassified_hap1.best_single.blobplot
blobtools view -i /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/BlobPlot_out/classified_Nui_plus_unclassified_hap1.best_single.blobplot.blobDB.json -o /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/BlobPlot_out/classified_Nui_plus_unclassified_hap1.best_single.blobplot
blobtools create -i /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/precontam/classified_Nui_plus_unclassified_hap2.purged.fa -b /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/bamfiles/classified_Nui_plus_unclassified_hap2_to_hifi.sorted.bam -t /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/blasthits/classified_Nui_plus_unclassified_hap2.vs.nt.best_single.1e10.megablast.out -o /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/BlobPlot_out/classified_Nui_plus_unclassified_hap2.best_single.blobplot
blobtools plot -i /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/BlobPlot_out/classified_Nui_plus_unclassified_hap2.best_single.blobplot.blobDB.json -o /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/BlobPlot_out/classified_Nui_plus_unclassified_hap2.best_single.blobplot
blobtools view -i /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/BlobPlot_out/classified_Nui_plus_unclassified_hap2.best_single.blobplot.blobDB.json -o /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/BlobPlot_out/classified_Nui_plus_unclassified_hap2.best_single.blobplot

EOF

Submitted batch job 2140870


# Use Rstudio to explore blob toolkit results

In [1]:
cp /workspace/hrasrb/kiwifruit_pangenome/Phasing/assemblies/ncbi_fcs_vs_bloptools.html /workspace/hrasrb/Repo/High-quality-genomes/Blueberry/Assembly_Contamination_Check

## Filter based on Blobtools filtering outputs

In [5]:
mkdir -p ${WKDIR}/postcontamremoval

In [9]:
cd $WKDIR

sbatch << EOF
#!/bin/bash
#SBATCH -J seqkit
#SBATCH -o ${LOG}/%J.out
#SBATCH -e ${LOG}/%J.err
#SBATCH --cpus-per-task=1
#SBATCH --mem=1G
#SBATCH --time=0:30:00

ml seqkit

cd ${WKDIR}/precontam

for assemb in *.fa

do
    tag=$(basename ${assemb} .purged.fa)
    seqkit grep --pattern-file ${WKDIR}/BlobPlot_out/${tag}_clean_contigs.txt -o ${WKDIR}/postcontamremoval/${tag}.clean.fa ${assemb}
done

EOF

Submitted batch job 2141649


In [10]:
scancel 2141649


In [9]:
cd /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/postcontamremoval

In [6]:
ls 

classified_M7_plus_unclassified_hap1.clean.fa
classified_M7_plus_unclassified_hap2.clean.fa
classified_Nui_plus_unclassified_hap1.clean.fa
classified_Nui_plus_unclassified_hap2.clean.fa
merged


In [11]:
module load BBMap/38.33
statswrapper.sh in=classified_M7_plus_unclassified_hap1.clean.fa,classified_M7_plus_unclassified_hap2.clean.fa,classified_Nui_plus_unclassified_hap1.clean.fa,classified_Nui_plus_unclassified_hap2.clean.fa format=3
#statswrapper.sh in=*.clean.fa format=3

module unload BBMap/38.33

java -ea -Xmx200m -cp /software/bioinformatics/BBMap-38.33/current/ jgi.AssemblyStatsWrapper format=3 in=classified_M7_plus_unclassified_hap1.clean.fa,classified_M7_plus_unclassified_hap2.clean.fa,classified_Nui_plus_unclassified_hap1.clean.fa,classified_Nui_plus_unclassified_hap2.clean.fa format=3
n_scaffolds	n_contigs	scaf_bp	contig_bp	gap_pct	scaf_N50	scaf_L50	ctg_N50	ctg_L50	scaf_N90	scaf_L90	ctg_N90	ctg_L90	scaf_max	ctg_max	scaf_n_gt50K	scaf_pct_gt50K	gc_avg	gc_std	filename
412	412	531169080	531169080	0.000	34	5202741	34	5202741	129	857712	129	857712	14681403	14681403	302	99.341	0.38543	0.03824	/powerplant/workspace/hraijc/Blueberry/Blueberry_trio/Assembly/postcontamremoval/classified_M7_plus_unclassified_hap1.clean.fa
378	378	522235882	522235882	0.000	31	5120845	31	5120845	125	799466	125	799466	17018523	17018523	306	99.535	0.38573	0.03227	/powerplant/workspace/hraijc/Blueberry/Blueberry_trio/Assembly/postcontamremoval/classified_M7_plus_unclassified_hap2.clean.fa
774	774	52286282

| n_scaffolds 	| n_contigs 	| scaf_bp 	| contig_bp 	| gap_pct 	| scaf_N50 	| scaf_L50 	| ctg_N50 	| ctg_L50 	| scaf_N90 	| scaf_L90 	| ctg_N90 	| ctg_L90 	| scaf_max 	| ctg_max 	| scaf_n_gt50K 	| scaf_pct_gt50K 	| gc_avg 	| gc_std 	| filename 	|
|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|
| 412 	| 412 	| 531169080 	| 531169080 	| 0.000 	| 34 	| 5202741 	| 34 	| 5202741 	| 129 	| 857712 	| 129 	| 857712 	| 14681403 	| 14681403 	| 302 	| 99.341 	| 0.38543 	| 0.03824 	| /powerplant/workspace/hraijc/Blueberry/Blueberry_trio/Assembly/postcontamremoval/classified_M7_plus_unclassified_hap1.clean.fa 	|
| 378 	| 378 	| 522235882 	| 522235882 	| 0.000 	| 31 	| 5120845 	| 31 	| 5120845 	| 125 	| 799466 	| 125 	| 799466 	| 17018523 	| 17018523 	| 306 	| 99.535 	| 0.38573 	| 0.03227 	| /powerplant/workspace/hraijc/Blueberry/Blueberry_trio/Assembly/postcontamremoval/classified_M7_plus_unclassified_hap2.clean.fa 	|
| 774 	| 774 	| 522862822 	| 522862822 	| 0.000 	| 34 	| 4551404 	| 34 	| 4551404 	| 172 	| 387370 	| 172 	| 387370 	| 19924815 	| 19924815 	| 466 	| 98.036 	| 0.38431 	| 0.02406 	| /powerplant/workspace/hraijc/Blueberry/Blueberry_trio/Assembly/postcontamremoval/classified_Nui_plus_unclassified_hap1.clean.fa 	|
| 483 	| 483 	| 532617838 	| 532617838 	| 0.000 	| 38 	| 3832205 	| 38 	| 3832205 	| 151 	| 684231 	| 151 	| 684231 	| 13744249 	| 13744249 	| 384 	| 99.369 	| 0.38458 	| 0.03650 	| /powerplant/workspace/hraijc/Blueberry/Blueberry_trio/Assembly/postcontamremoval/classified_Nui_plus_unclassified_hap2.clean.fa 	|


In [13]:
sbatch << EOF
#!/bin/bash
#SBATCH -J BUSCO
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=12
#SBATCH --mem=10G
#SBATCH --time=21:00:00

module load BUSCO/v5.2.2 

cd /workspace/hraijc/Blueberry/Blueberry_trio/Assembly/postcontamremoval

busco -m geno -c 12 -i classified_M7_plus_unclassified_hap1.clean.fa -l eudicots_odb10 -o classified_M7_plus_unclassified_hap1_busco
busco -m geno -c 12 -i classified_M7_plus_unclassified_hap2.clean.fa -l eudicots_odb10 -o classified_M7_plus_unclassified_hap2_busco
busco -m geno -c 12 -i classified_Nui_plus_unclassified_hap1.clean.fa -l eudicots_odb10 -o classified_Nui_plus_unclassified_hap1_busco
busco -m geno -c 12 -i classified_Nui_plus_unclassified_hap2.clean.fa -l eudicots_odb10 -o classified_Nui_plus_unclassified_hap2_busco

EOF

Submitted batch job 2706273
